# Implementing word2vec from scratch

In this notebook we will implement word2vec (skipgram method) from scratch using numpy. To see detailed notes, go to lecture 1 notebook of CS224N - [Lecture-1-Introduction-and-Word-Vectors](Lecture-1-Introduction-and-Word-Vectors.ipynb). Here we jump directly to code.

As a first step, let's first generate the training data we are going to use train the word2vec.

## Variables

We will use the following variables in our implementation of `word2vec`.

- `V` : Number of unique words in the corpus (vocabulary)
- `x` : Input layer (One hot encoding of our input word)
- `N` : Number of neurons in the hidden layer of neural network i.e. the dimension of the vector representation of the word.
- `W` : Weights between input layer and hidden layer.
- `W'`: Weights between hidden layer and output layer.
- `y` : A softmax layer having probabilities of each word in our corpus

![](https://firebasestorage.googleapis.com/v0/b/firescript-577a2.appspot.com/o/imgs%2Fapp%2Fdailygrind%2FiX7rd0bVNJ.png?alt=media&token=5ac14c23-1165-4a20-be6c-c1dac015d50d)



## Implementation details

- Input Layer (`x`): It is one hot encoding of the current center word. It's size of is *V* x 1, where *V* is the size of the vocabulary/unique words in corpus.
- Weight Matrix (`W`) : It is the matrix of the representation of the all center words in the vocabulary. There will be *V* rows (one for each word in vocabulary) and each row vector has N dimension, hence its size is V x N.
- Hidden Layer (`h`): It is nothing but representation of the center word in column vector. It is calculated by 
    $$
        h = W^Tx
    $$
- Weight Matrix 2 (`W'`): It is the matrix which stores the context word representation of each word in vocabulary. However, here each word is represented as column. There will be *V* columns of *N* dimension for each word in vocabulary. Thus shape will be N x V.

- Output Layer (`y`): This is where we calculate similarity between center word with the chosen context word, and then calculate softmax of the similarity (nothing but cosine similarity). The softmax converts the integer values to probability distribution.

- Forward propagation: The process of moving getting the output layer `y` from `x` is forward propagation.
- Backward propagation: In this process, based on the loss we calculate using gradient descent, we propagate the loss values in backward direction and update the `W` and `W'` in the process.




## Generating training data.

As described in the lecture notebook, the idea is to go through the corpus and treat each token as center word, and tokens around it as context words. We are not going to use fancy methods of tokenization here, but using those, the word2vec can perform amazing. We also maintain the two dictionaries to map word to index and index to word (This is the how we represent o as index of outside word and c as index of center word and map it to j in the index formula) 

In [6]:
import numpy as np
from collections import defaultdict

def generate_training_data(corpus, context_size):
    word_counts = defaultdict(int);
    tokens = []
    X, y = [], []
    for sentence in corpus:
        word_list = sentence.split()
        for word in word_list:
            word = word.lower()
            word_counts[word] += 1
            tokens.append(word)
    N = len(word_counts.keys())
    words_list = sorted(list(word_counts.keys()), reverse=False)
    word_index = dict((word, i) for (i, word) in enumerate(words_list))
    index_word = dict((i, word) for (i, word) in enumerate(words_list))
    
    for i in range(N):
        context_indices = list(range(max(0, i - context_size), i)) + \
            list(range(i+1, min(i + context_size + 1, N)))
        for j in context_indices:
            X.append(word_index[tokens[i]])
            y.append(word_index[tokens[j]])
        
    X = np.array(X)
    X = np.expand_dims(X, axis=0)
    y = np.array(y)
    y = np.expand_dims(y, axis=0)
    
    return (X,y, word_index, index_word)

Let's just briefly test this

In [5]:
corpus = ['''You could never convince a monkey to give you a banana by promising him limitless bananas after death in monkey heaven''',
'''History is something that very few people have been doing while everyone else was ploughing fields and carrying water buckets.''']
(X, y, word_index, index_word) = generate_training_data(corpus, context_size=3)
vocab_size = len(index_word)
print("Vocabulary size:", vocab_size)
print("Shape of training data: ", X.shape, y.shape)
print("Word to index: ", word_index)

38
Vocabulary size: 38
Shape of training data:  (1, 216) (1, 216)
Word to index:  {'a': 0, 'after': 1, 'and': 2, 'banana': 3, 'bananas': 4, 'been': 5, 'buckets.': 6, 'by': 7, 'carrying': 8, 'convince': 9, 'could': 10, 'death': 11, 'doing': 12, 'else': 13, 'everyone': 14, 'few': 15, 'fields': 16, 'give': 17, 'have': 18, 'heaven': 19, 'him': 20, 'history': 21, 'in': 22, 'is': 23, 'limitless': 24, 'monkey': 25, 'never': 26, 'people': 27, 'ploughing': 28, 'promising': 29, 'something': 30, 'that': 31, 'to': 32, 'very': 33, 'was': 34, 'water': 35, 'while': 36, 'you': 37}


We will need each word to be represented as one-hot encoding representation. So let's do that too.

In [14]:
def word2OneHot(word, word_index):
    vocab_size = len(word_index)
    y_one_hot = np.zeros((vocab_size))
    y_one_hot[word_index[word]] = 1
    return y_one_hot

In [13]:
word2OneHot('monkey', word_index)

(38,)


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])